In [1]:
from typing import List
import networkx as nx
import numpy as np
import pandas as pd
import datetime
from hatch import contributions_to_token_batches, TokenBatch, Commons
from convictionvoting import trigger_threshold
from network_helpers import *
from IPython.core.debugger import set_trace

from cadCAD.configuration import Configuration


def initialize_network_of_hatchers(participants: List[TokenBatch]) -> nx.DiGraph:
    """
    The role of this function is simply to fill up the directed graph.
    Helper functions setting up the tokens etc should be defined elsewhere
    and only provide values to this dumb function to put in the graph.
    """
    network = nx.DiGraph()
    for i, p in enumerate(participants):
        # Note: TokenBatch arguments don't mean anything if they're nonvesting
        network.add_node(i, type="participant", holdings_vesting=p, holdings_nonvesting=TokenBatch(0, 5, 5), sentiment=np.random.rand())
    return network


# The cell below is for copy/pasting policy functions from libraries into, for debugging

In [2]:
# contributions = [5e5, 5e5, 2.5e5]
contributions = [np.random.rand() * 10e5 for i in range(60)]
token_batches, initial_token_supply = contributions_to_token_batches(contributions, 0.1, 60)

network = initialize_network_of_hatchers(token_batches)
commons = Commons(sum(contributions), initial_token_supply, exit_tribute=0.35)
network = add_proposals_and_relationships_to_network(network, 3, commons._funding_pool, commons._token_supply)

initial_conditions = {
    "network": network,
    "commons": commons,
    "funding_pool": commons._funding_pool,
    "collateral_pool": commons._collateral_pool,
    "token_supply": commons._token_supply,
    "sentiment": 0.5,
}

partial_state_update_blocks = [
    {
        "policies": {
            "gen_new_participants_proposals_funding": gen_new_participants_proposals_funding_randomly
        },
        'variables': {
            'network': add_participants_proposals_to_network,
            'commons': new_participants_and_new_funds_commons,
        }
    },
    {
        "policies": {},
        "variables": {
            "funding_pool": update_funding_pool,
            "collateral_pool": update_collateral_pool,
            "token_supply": update_token_supply,
        }
    },
    {
        "policies": {
            "make_active_proposals_complete_or_fail": make_active_proposals_complete_or_fail_randomly,
        },
        "variables": {
            "sentiment": sentiment_decays_wo_completed_proposals,
            "network": update_network_w_proposal_status,
        }
    },
    {
        "policies": {
            "decide_proposals_to_fund": conviction_gathering,
        },
        "variables": {
            'commons': decrement_commons_funding_pool, #funds expended
            'sentiment': update_sentiment_on_release, #releasing funds can bump sentiment
            'network': update_proposals #reset convictions, and participants sentiments
                                        #update based on affinities
        }
    },
    {
        "policies": {
            "participants_start_to_act": participants_buy_more_if_they_feel_good_and_vote_for_proposals
        },
        "variables": {
            "network": update_holdings_nonvesting_of_participants,
        }
    },
    {
        "policies": {},
        "variables": {
            "funding_pool": update_funding_pool,
            "collateral_pool": update_collateral_pool,
            "token_supply": update_token_supply,
        }
    },
]

In [3]:
simulation_parameters = {
    'T': range(150),
    'N': 1,
    'M': {
        "sentiment_decay": 0.01, #termed mu in the state update function
        "trigger_threshold": trigger_threshold,
        "min_proposal_age_days": 7, # minimum periods passed before a proposal can pass,
        "sentiment_sensitivity": 0.75
    }
}

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# The configurations above are then packaged into a `Configuration` object
config = Configuration(initial_state=initial_conditions, #dict containing variable names and initial values
                       partial_state_update_blocks=partial_state_update_blocks, #dict containing state update functions
                       sim_config=simulation_parameters #dict containing simulation parameters
                      )


from cadCAD.engine import ExecutionMode, ExecutionContext, Executor
exec_mode = ExecutionMode()
exec_context = ExecutionContext(exec_mode.single_proc) # Do not use multi_proc, breaks ipdb.set_trace()
executor = Executor(exec_context, [config]) # Pass the configuration object inside an array
raw_result, tensor = executor.execute() # The `execute()` method returns a tuple; its first elements contains the raw results


                            __________   ____ 
          ________ __ _____/ ____/   |  / __ \
         / ___/ __` / __  / /   / /| | / / / /
        / /__/ /_/ / /_/ / /___/ ___ |/ /_/ / 
        \___/\__,_/\__,_/\____/_/  |_/_____/  
        by BlockScience
        
Execution Mode: single_proc: [<cadCAD.configuration.Configuration object at 0x7f9a38e62d30>]
Configurations: [<cadCAD.configuration.Configuration object at 0x7f9a38e62d30>]
{'new_participant': False, 'new_participant_investment': 0, 'new_participant_tokens': 0, 'new_proposal': False, 'funds_arrival': 30.69794781542132}
{'completed': [], 'failed': []}
{'accepted': [], 'triggers': {60: 188830679.67969963, 61: 175035849.27167666, 62: 177245801.1865877}}
{'delta_holdings': {0: 0, 1: 0, 2: 0, 3: -0.219163893641037, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0.0338582542966812, 19: 0, 20: 0, 21: 0, 22: 0, 23: -0.025882648091218517, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30

TypeError: cannot convert dictionary update sequence element #0 to a sequence

In [ ]:
df = pd.DataFrame(raw_result)
df_final = df[df.substep.eq(2)]


In [ ]:
df_final.plot("timestep", "collateral_pool", grid=True)
df_final.plot("timestep", "token_supply", grid=True)
df_final.plot("timestep", "funding_pool", grid=True)

In [ ]:
import matplotlib.pyplot as plt
supporters = get_edges_by_type(network, 'support')
influencers = get_edges_by_type(network, 'influence')
competitors = get_edges_by_type(network, 'conflict')

nx.draw_kamada_kawai(network, nodelist = get_participants(network), edgelist=influencers)
plt.title('Participants Social Network')